In [1]:
import numpy as np
import pandas as pd
from collections import Counter
from math import sqrt, pi

# https://docs.astropy.org/en/stable/api/astropy.coordinates.SkyCoord.html
from astropy.coordinates import SkyCoord, EarthLocation, AltAz, Angle, ICRS, FK5
# https://astroquery.readthedocs.io/en/latest/vizier/vizier.html
from astroquery.vizier import Vizier
from astropy.time import Time
from astropy import units as u

gzip was not found on your system! You should solve this issue for astroquery.eso to be at its best!
On POSIX system: make sure gzip is installed and in your path!On Windows: same for 7-zip (http://www.7-zip.org)!


# Coordinates to look at

In [2]:
def get_viewpoints(obstime=Time('2020-4-11 20:00:00'), location=EarthLocation.of_address("Dublin, Ireland")):
    """ For a given location and time, return astropy AltAz points for 9 areas of interest
    We've 9 target areas to look at. 
    zenith, then at 45degrees altitude N, NE, E, SE, S, SW, W & NW
    """
    points=[(0,90), (0,45), (45,45), (90,45), (135,45), (180,45), (225,45), (270,45), (315,45)]

    return [ AltAz(obstime=obstime, 
                   location=location, 
                   az=Angle('{0}d'.format(x[0])), 
                   alt=Angle('{0}d'.format(x[1]))) 
            for x in points]
    


# Work out the number of stars in a given circular are a of sky. 

In [3]:
from astropy import units as u
from math import sqrt, pi
# https://docs.astropy.org/en/stable/api/astropy.coordinates.SkyCoord.html
from astropy.coordinates import SkyCoord
# https://astroquery.readthedocs.io/en/latest/vizier/vizier.html
from astroquery.vizier import Vizier


In [4]:
def rsterd(st):
    """ Get radius in degress of a steradian area
    """
    return sqrt(0.09/pi) * (180/pi)

r = rsterd(0.09)

# Is 0.09 steradians roughly a circle of 9.6 degrees radius? Think so...

In [5]:
# I/259 is the Tyco-2 catalog
# the +_r adds a column to tell us how far the star is from the target

v = Vizier(columns=["*", "+_r"], catalog="I/259")

# Fow limit needs to be set to -1 as otherwise it's limited to 50 results
v.ROW_LIMIT = -1

# We can query about a named object:
#result = v.query_region("M1", radius='{0}d'.format(r))
#result = v.query_region(point, radius='{0}d'.format(r))

# Or around a coordinate (SkyCoord can handle hms or decimals):
#result = v.query_region(SkyCoord('0h39m15.9s', '0d53m17.016s', frame='icrs'), radius='{0}d'.format(r))


In [6]:
def generate_result_row(p):
    """ For a given point p, return a datarfame row with a summary of the stars found with Vmag < 7.5
    """
    mags=np.arange(-1,9)
    df = pd.DataFrame(columns = mags) 
    
    # This transform is ridiculously slow.
    icrs = p.transform_to(ICRS)
    # query_region is pretty slow too...
    result = v.query_region(p, radius='{0}d'.format(r))
    # debug
    # print(result[0])
    t = result[0]
    mask = t['VTmag'] < 7.5  # Makes a boolean selection mask (numpy array) and gets only stars brighter than mag 7
    t_new = t[mask]
    df = df.append(Counter([int(x) for x in t_new['VTmag']]), ignore_index=True).fillna(0)

    df1 = pd.DataFrame(columns=["Time", "lat", "lon", "az", "alt", "ra", "dec"])
    df1 = df1.append(pd.Series({"Time" : p.obstime,
                           "lat" : p.location.lat,
                           "lon" : p.location.lon,
                           "az" : p.az,
                           "alt" : point.alt,
                           "ra" : icrs.ra,
                           "dec" : icrs.dec}), ignore_index=True)  
    
    dfrow = pd.concat([df1, df], axis=1, sort=False)
    return dfrow

In [ ]:
# Slow...
resultdf = pd.DataFrame()
for point in get_viewpoints(obstime=Time('2020-4-11 20:00:00')):
    resultdf = pd.concat([resultdf, generate_result_row(point)])
for point in get_viewpoints(obstime=Time('2020-4-18 20:00:00')):
    resultdf = pd.concat([resultdf, generate_result_row(point)])
for point in get_viewpoints(obstime=Time('2020-4-25 20:00:00')):
    resultdf = pd.concat([resultdf, generate_result_row(point)])
for point in get_viewpoints(obstime=Time('2020-5-2 20:00:00')):
    resultdf = pd.concat([resultdf, generate_result_row(point)])
for point in get_viewpoints(obstime=Time('2020-5-9 20:00:00')):
    resultdf = pd.concat([resultdf, generate_result_row(point)])

In [8]:
# Tidy up the dataframe
try:
    resultdf.rename(columns={-1: "-1-0",
                        0 : "0-1",
                        1 : "1-2",
                        2 : "2-3",
                        3 : "3-4",
                        4 : "4-5",
                        5 : "5-6",
                        6 : "6-7",
                        7 : "7-8"},
               inplace=True)
    resultdf.drop(columns=[8], inplace=True)
except KeyError as e:
    pass
resultdf.to_csv('starsummary.csv')

In [9]:
resultdf

,Time,lat,lon,az,alt,ra,dec,-1-0,0-1,1-2,2-3,3-4,4-5,5-6,6-7,7-8
0,2020-04-11 20:00:00.000,53d20m59.1522s,-6d15m36.9835s,0d00m00s,90d00m00s,133d51m47.058s,53d25m27.0667s,0.0,0.0,0.0,0.0,5.0,5.0,16.0,56.0,53.0
0,2020-04-11 20:00:00.000,53d20m59.1522s,-6d15m36.9835s,0d00m00s,45d00m00s,314d31m26.6912s,81d34m40.1691s,0.0,0.0,0.0,1.0,1.0,7.0,23.0,70.0,75.0
0,2020-04-11 20:00:00.000,53d20m59.1522s,-6d15m36.9835s,45d00m00s,45d00m00s,221d41m34.1234s,60d03m22.2377s,0.0,0.0,0.0,0.0,2.0,6.0,17.0,52.0,63.0
0,2020-04-11 20:00:00.000,53d20m59.1522s,-6d15m36.9835s,90d00m00s,45d00m00s,193d09m02.0739s,34d40m18.5365s,0.0,0.0,0.0,1.0,0.0,10.0,14.0,44.0,36.0
0,2020-04-11 20:00:00.000,53d20m59.1522s,-6d15m36.9835s,135d00m00s,45d00m00s,165d14m02.2738s,15d42m14.0077s,0.0,0.0,0.0,1.0,2.0,6.0,13.0,32.0,41.0
0,2020-04-11 20:00:00.000,53d20m59.1522s,-6d15m36.9835s,180d00m00s,45d00m00s,133d57m35.2171s,8d25m41.9182s,0.0,0.0,0.0,0.0,3.0,5.0,21.0,57.0,66.0
0,2020-04-11 20:00:00.000,53d20m59.1522s,-6d15m36.9835s,225d00m00s,45d00m00s,102d40m17.5237s,15d37m14.6822s,0.0,0.0,0.0,0.0,5.0,9.0,31.0,103.0,119.0
0,2020-04-11 20:00:00.000,53d20m59.1522s,-6d15m36.9835s,270d00m00s,45d00m00s,74d44m16.4078s,34d31m59.7394s,0.0,0.0,0.0,1.0,3.0,6.0,27.0,90.0,72.0
0,2020-04-11 20:00:00.000,53d20m59.1522s,-6d15m36.9835s,315d00m00s,45d00m00s,46d14m34.9224s,59d53m37.8453s,0.0,0.0,0.0,1.0,2.0,9.0,29.0,83.0,93.0
0,2020-04-18 20:00:00.000,53d20m59.1522s,-6d15m36.9835s,0d00m00s,90d00m00s,140d46m31.8063s,53d25m59.4414s,0.0,0.0,0.0,0.0,5.0,4.0,15.0,52.0,51.0
